In [1]:
import chardet
with open('huge_train_tweets.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(1000000))
result

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [2]:
#Libraries
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

train = pd.read_csv('huge_train_tweets.csv')
columns = train.columns.values
print(columns)
#train.head()

['target' 'id' 'date' 'flag' 'user' 'text ']


In [3]:
mean = train['target'].mean()
binary_target = []

for element in train['target']:
    if element >= mean:
        binary_target.append(1)
    elif element < mean:
        binary_target.append(0)
train['target'] = binary_target
train.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
def form_sentence(tweet):
    tweet_blob = TextBlob(tweet)
    return ' '.join(tweet_blob.words)
print(form_sentence(train['text '].iloc[10]))
print(train['text '].iloc[10])

spring break in plain city it 's snowing
spring break in plain city... it's snowing 


In [5]:
#Removes words and symbols with little to no 
#sentimental value
def clean_tweets(tweet):
    tweet_split = [wrd for wrd in tweet.split()]
    clean_wrds = [s for s in tweet_split if re.match(r'[^\W\d]*$', s)]
    clean_s = ' '.join(clean_wrds)
    clean = [word for word in clean_s.split() if word.lower() not in stopwords.words('english')]
    return clean

In [6]:
#Normalizes the tweets
#eg.replaces words with same meaning
def normalize(tweets):
    lem = WordNetLemmatizer()
    normalized_tweet = []
    for word in tweets:
        normalized_text = lem.lemmatize(word,'v')
        normalized_tweet.append(normalized_text)
    return normalized_tweet

In [7]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB()),
])

In [10]:
tweet_train, tweet_test, label_train, label_test = train_test_split(train['text '], train['target'], test_size=0.2)
pipeline.fit(tweet_train,label_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [ ]:
predictions = pipeline.predict(tweet_test)
print(classification_report(predictions,label_test))
print(accuracy_score(predictions, label_test))

print(predictions)

In [14]:
new_tweets = pd.read_csv('train_tweets.csv')
new_predictions = pipeline.predict(new_tweets['tweet'])
print(accuracy_score(new_predictions, new_tweets['label']))

0.3630874163068644


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
